In [9]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from ipywidgets import interact, FloatSlider
from scipy.stats import norm

# Kou Double-Exponential Jump model

$$ \frac{dS_t}{S_t} = (r - q - \lambda \kappa) dt + \sigma dW_t + (e^Y - 1)dN_t $$

- N_t: Poisson process with intensity $\lambda$
- Y: jump size (log-jump) with double-exponential density

$$ f_Y(y) = \begin{cases}
p \eta_+e^{-\eta_+y}, & y \ge 0 \\
(1-p) \eta_-e^{-\eta_-y},  & y < 0
\end{cases} $$

$$ \kappa = \mathbb{E}\!\left[e^{Y} - 1\right]
= \mathbb{E}\!\left[e^{Y}\right] - 1
= \left( \frac{p\,\eta_{+}}{\eta_{+} - 1}
      + \frac{(1 - p)\,\eta_{-}}{\eta_{-} + 1} \right) - 1 $$

In [10]:
@dataclass
class KouParams:
    lamda: float
    sigma: float
    prob: float
    eta_plus: float
    eta_minus: float

def kou_cf(u, S0, T, r, q, p:KouParams):
    x0 = np.log(S0)
    kappa = (p.prob * p.eta_plus) / (p.eta_plus - 1.0) + (1 - p.prob) * p.eta_minus / (p.eta_minus + 1.0) - 1.0

    M_Y = p.prob * p.eta_plus / (p.eta_plus - 1j * u) + (1 -p.prob) * p.eta_minus / (p.eta_minus + 1j * u)

    drift = x0 + (r - q - p.lamda * kappa - 0.5 * p.sigma ** 2) * T

    return np.exp(1j * u * drift - 0.5 * p.sigma ** 2 * u ** 2 * T + p.lamda * T * (M_Y - 1.0))


def simpson_weights(N: int):
    if N % 2 != 0:
        raise ValueError("N must be even")
    w = np.ones(N)
    w[1:N-1:2] = 4
    w[2:N-2:2] = 2
    return w

# alpha dampening factor
# eta is step in frequency domain

def kou_fft_calls(S0:float, T:float, r:float, q:float, p:KouParams, N:int = 2**14, eta:float = 0.1, alpha:float = 1.5):
    n = np.arange(N)
    v = eta * n   
    i = 1j
    phi_shift = kou_cf(v - (alpha + 1)*i, S0, T , r, q, p)        
    denom = (alpha ** 2 + alpha  - v ** 2 + i*(2*alpha + 1)* v)  
    psi = np.exp(-r * T) * phi_shift/denom

    w = simpson_weights(N) * (eta / 3.0)

    # FFT coupling
    lam  = 2.0 * np.pi / (N * eta)   # Δk (log-strike step)
    b = 0.5 * N * lam     # half-width in k
    x   = psi * np.exp(1j * b * v) * w

    F  = np.fft.fft(x)
    F  = np.real(F)

    j = np.arange(N)
    k = -b + j * lam                 # k = ln K
    K = np.exp(k)

    calls = np.exp(-alpha * k) / np.pi * F
    order = np.argsort(K)
    return K[order], np.maximum(calls[order], 0.0)


def kou_fft_call_price(S0: float, K: float, T: float, r: float, q: float, p: KouParams,
    N: int = 2**14, eta: float = 0.1, alpha: float = 1.5):
    K_grid, C_grid = kou_fft_calls(S0, T, r, q, p, N=N, eta=eta, alpha=alpha)
    if K <= K_grid[0]:
        return C_grid[0]
    if K >= K_grid[-1]:
        return C_grid[-1]
    idx = np.searchsorted(K_grid, K)
    x0, x1 = K_grid[idx-1], K_grid[idx]
    y0, y1 = C_grid[idx-1], C_grid[idx]
    return y0 + (y1 - y0) * (K - x0) / (x1 - x0)

def kou_fft_put_price(
    S0: float, K: float, T: float, r: float, q: float, p: KouParams,
    N: int = 2**14, eta: float = 0.1, alpha: float = 1.5
):
    """Put via put–call parity."""
    C = kou_fft_call_price(S0, K, T, r, q, p, N=N, eta=eta, alpha=alpha)
    return C - S0*np.exp(-q*T) + K*np.exp(-r*T)

In [11]:
def European_Call(S0, K, T, r , q, sigma):
    d1 = (np.log(S0/K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return  S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def bs_vega(S0, K, T, r, q, sigma):
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S0 * norm.pdf(d1) * np.sqrt(T)

def get_implied_vol(market_price, S0, K, T, r, q, max_iter = 200, tolerance = 1e-05):
    sigma = 0.3
    for _ in range(max_iter):
        bs_price = European_Call(S0, K, T, r, q, sigma)
        diff = bs_price - market_price
        if abs(diff) < tolerance:
            return sigma
        vega = bs_vega(S0,K,T,r, q, sigma)
        if vega == 0:
            return None
        sigma -= diff/vega
    return None

In [19]:

def plot_BS_implied_vol(S0, T, r, q, sigma, lamda, prob, eta_plus, eta_minus):
    K_min = S0 * 0.5
    K_max = S0 * 2
    fig, ax = plt.subplots(1,2, figsize = (12,6))
    
    hp = KouParams(sigma = sigma, lamda = lamda, prob = prob, eta_plus = eta_plus, eta_minus= eta_minus)
    moneyness_grid = np.log(np.linspace(K_min , K_max, 300)/(S0 * np.exp(r * T)))
    K_grid = np.linspace(K_min, K_max, 300)
   
    
    kou_prices = [kou_fft_call_price(S0, k, T, r , q, hp) for k in K_grid]
    black_scholes_prices = [European_Call(S0, k, T, r, q, sigma) for k in K_grid]
    BS_implied_vols = [get_implied_vol(merton_price, S0, strike, T, r, q) for strike, merton_price in zip(K_grid,kou_prices)] 
    
    ax[0].scatter(K_grid, BS_implied_vols, marker = "x", color = 'black')
    ax[0].set_xlabel("Strike")
    ax[0].set_ylabel("BS implied volatility (%)")
    ax[0].set_title("BS implied volatility(%) for Kou model")
    ax[0].grid(True)
    ax[1].scatter(K_grid, kou_prices, marker = "x", color = 'black')
    ax[1].plot(K_grid, black_scholes_prices, color = 'blue')
    ax[1].set_xlabel("Strike")
    ax[1].set_ylabel("Call option price")
    ax[1].set_title("Call option price for Kou model")
    ax[1].grid(True)
    plt.show()


interact(plot_BS_implied_vol,
         S0=FloatSlider(value=100, min=20,  max=300, step=1,   description="S0"),
         T=FloatSlider(value=0.4, min=0.05, max=5.0, step=0.05, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         q=FloatSlider(value=0.0, min=0.0, max=0.1, step=0.002, description="q"),
         sigma=FloatSlider(value=0.2, min=0.01, max=10.0, step=0.01, description="sigma"),
         lamda=FloatSlider(value=0.4, min=0.001, max=3.0, step=0.005, description="lamda"),
         prob=FloatSlider(value=0.3, min=0.01, max=0.99, step=0.005, description="p"),
         eta_plus=FloatSlider(value=10.0, min=1.1, max=100.0, step=0.005, description="eta_plus"),
         eta_minus=FloatSlider(value=4.0, min=1.0, max=100.0, step=0.005, description="eta_minus")

        )
       

interactive(children=(FloatSlider(value=100.0, description='S0', max=300.0, min=20.0, step=1.0), FloatSlider(v…

<function __main__.plot_BS_implied_vol(S0, T, r, q, sigma, lamda, prob, eta_plus, eta_minus)>